In [16]:
import pandas as pd
import numpy as np

In [17]:
df = pd.read_csv('couples.unique.csv')
df = df.set_index('Couple')
df = df[~pd.isnull(df['spvl'])]
og_df = df.copy()

## Options of how to treat the data

In [18]:
base_output_name = 'blanquart.couples.for.fitting'

file_suffix = ''
dose_column = 'spvl'
# remove undectable viral load?
remove_undectable = True
if remove_undectable:
    file_suffix += '.ur'
    dose_column = 'spvl.ur'

In [19]:
if remove_undectable:
    df = df[~pd.isna(df['spvl.ur'])]

## Processing data

In [20]:
df['partner.ever.seroconverted'] = (df['partner.firstPosDate'] != np.inf)
df['partner.ever.seroconverted'].sum()

225

In [21]:
# do this later with inferred seroconversion date?
df['partner.first.pos.after.art'] = df['partner.ever.seroconverted'] & (df['partner.firstPosDate'] > df['index.first.art.date'])
df['partner.first.pos.after.art'].sum()

17

In [22]:
df['partner.seroconverted.before.art'] = df['partner.ever.seroconverted'] & ~df['partner.first.pos.after.art']
df['partner.seroconverted.before.art'].sum()

208

In [23]:
df['index.inferred.spvl.start.date'] = df['index.firstPosDate'] + 0.5

In [24]:
# here we can choose to do midpoint or not!

df['partner.inferred.seroconversion.date'] = np.where(
    df['partner.ever.seroconverted'],
    (df['partner.firstPosDate'] + df['partner.lastNegDate'])/2,
    np.inf
)

In [25]:
df['infectious.contact.period.end'] = np.where(
    df['partner.seroconverted.before.art'],
    df[['index.first.art.date', 'partner.inferred.seroconversion.date']].min(axis=1),
    df[['index.first.art.date', 'partner.lastNegDate']].min(axis=1),
)

In [26]:
df['duration'] = df['infectious.contact.period.end'] - df['index.inferred.spvl.start.date']

In [27]:
df[df['duration'] <= 0].index

Index([ 219,  255,  279,  291,  320,  392,  466,  514,  591,  629,  685,  703,
        705,  722,  737,  781,  847,  859,  866,  882,  899,  901,  921, 1207,
       1219, 1343, 1362, 1430, 1458, 1486, 1640, 1758, 1855, 1860, 1867, 1989,
       2098, 2123, 2135, 2137, 2144, 2218, 2241, 2367, 2390, 2422, 2506, 2520,
       2604, 2803, 2845, 2860, 2867, 2948],
      dtype='int64', name='Couple')

In [28]:
df_bad = df[df['duration'] <= 0]
df = df[df['duration'] > 0]

with open('couples.extra.columns.csv', 'w') as f:
    df.to_csv(f)

df_new = pd.DataFrame({
    'couple': df.index,
    'dose': 10**df[dose_column],
    'number': 1,
    'success': df['partner.seroconverted.before.art'],
    'duration': df['duration'],
    'dose_frequency': 9*12.
    },
)

df_new['success'] = df_new['success'].astype(int)
df_new = df_new.set_index('couple')

with open(base_output_name + file_suffix + '.csv', 'w') as f:
    df_new.to_csv(f)

with open('dropped.blanquart.couples.csv', 'w') as f:
    df_bad.to_csv(f)

In [29]:
df_bad[df_bad['index.first.art.date'] < df_bad['index.firstPosDate']]

,male.firstPosDate,male.lastNegDate,female.firstPosDate,female.lastNegDate,male.index,female.index,female.firstObsDate,male.firstObsDate,female.lastObsDate,male.lastObsDate,...,assay,n.vl.ur,spvl.ur,partner.ever.seroconverted,partner.first.pos.after.art,partner.seroconverted.before.art,index.inferred.spvl.start.date,partner.inferred.seroconversion.date,infectious.contact.period.end,duration
Couple,,,,,,,,,,,,,,,,,,,,,
703,inf,2014.1,2010.4,NaN,False,True,2010.4,2010.4,2012.1,2012.1,...,Roche 1.5,1,5.449969,False,False,False,2010.9,inf,2009.1,-1.8
705,2008.5,NaN,inf,2011.6,True,False,2008.5,2008.5,2011.6,2011.6,...,Roche 1.5,1,4.822312,False,False,False,2009.0,inf,2005.0,-4.0
921,inf,2010.4,2007.5,NaN,False,True,2007.5,2007.5,2010.4,2010.4,...,Roche 1.5,1,3.972851,False,False,False,2008.0,inf,2005.9,-2.1
2604,2008.6,1995.3,inf,2013.9,True,False,2008.6,2008.6,2011.7,2011.7,...,Roche 1.5,1,5.249543,False,False,False,2009.1,inf,2007.6,-1.5
2860,inf,2010.9,2007.7,NaN,False,True,2007.7,2007.7,2010.9,2010.9,...,Roche 1.5,1,4.628552,False,False,False,2008.2,inf,2005.8,-2.4


In [30]:
df

,male.firstPosDate,male.lastNegDate,female.firstPosDate,female.lastNegDate,male.index,female.index,female.firstObsDate,male.firstObsDate,female.lastObsDate,male.lastObsDate,...,assay,n.vl.ur,spvl.ur,partner.ever.seroconverted,partner.first.pos.after.art,partner.seroconverted.before.art,index.inferred.spvl.start.date,partner.inferred.seroconversion.date,infectious.contact.period.end,duration
Couple,,,,,,,,,,,,,,,,,,,,,
1,2002.1,2000.8,1994.9,NaN,False,True,1994.9,1994.9,2002.1,2002.1,...,Roche 1.0,4,3.085649,True,False,True,1995.4,2001.45,2001.45,6.05
7,inf,1995.7,1994.9,NaN,False,True,1994.9,1994.9,1995.7,1995.7,...,Roche 1.0,1,2.750508,False,False,False,1995.4,inf,1995.70,0.30
9,2002.1,2000.8,inf,2007.9,True,False,2002.1,2002.1,2003.3,2002.1,...,Roche 1.0,1,5.669317,False,False,False,2002.6,inf,2003.30,0.70
11,inf,1996.5,1994.9,NaN,False,True,1994.9,1994.9,1996.5,1996.5,...,Roche 1.0,5,3.595304,False,False,False,1995.4,inf,1996.50,1.10
14,1994.9,NaN,inf,2000.9,True,False,1999.8,1999.8,2000.9,2000.7,...,Roche 1.0,1,3.327359,False,False,False,1995.4,inf,2000.90,5.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3111,inf,2008.2,1995.4,NaN,False,True,2004.8,2004.9,2008.2,2008.2,...,Roche 1.5,1,5.745893,False,False,False,1995.9,inf,2005.20,9.30
3114,inf,2009.1,2005.6,NaN,False,True,2005.6,2005.6,2009.1,2009.1,...,Roche 1.5,1,2.966611,False,False,False,2006.1,inf,2007.30,1.20
3117,inf,2013.5,2002.6,NaN,False,True,2006.7,2006.7,2011.6,2011.6,...,Roche 1.5,1,3.723456,False,False,False,2003.1,inf,2005.20,2.10
